# Deploying `sktime` via `MLflow` & `MLflavors`

requires MLflow using the [MLflavors library](https://github.com/ml-toolkits/mlflavors).

**Notebook contents**:

- saving `sktime` models as MLflow artifacts.
- loading `sktime` models from MLflow for batch inference.
- deploying `sktime` models to a serving endpoint using MLflow

Design summary:

* uses `pyfunc` based custom flavor similar to `sktime` example in [MLflow documentaion](https://mlflow.org/docs/latest/models.html#custom-flavors)
* single-row `pandas` `DataFrame` configuration arguments to address the `sktime` prediction API in inference step

## Saving `sktime` model as an MLflow artifact

Example: save fitted model, model parameters, and results of this experiment to server

* fit `NaiveForecaster` on longley data (with exogenous vars)
* evaluate via MAE and MAPE

first without mlflow

In [1]:
from sktime.datasets import load_longley
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import (
    mean_absolute_error,
    mean_absolute_percentage_error,
)

y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

forecaster = NaiveForecaster()
forecaster.fit(
    y_train,
    X=X_train,
    fh=[1, 2, 3, 4],
)

# Extract parameters
parameters = forecaster.get_params()

# Evaluate model
y_pred = forecaster.predict(X=X_test)
metrics = {
    "mae": mean_absolute_error(y_test, y_pred),
    "mape": mean_absolute_percentage_error(y_test, y_pred),
}


with `mlflow` / `mlflavors`:

* use `mlflow` context manager `start_run`
* results are logged/saved using standard `mlflow.log_params`, `log_metrics`
* model is logged/saved using `mlflavors.sktime.log_model`

for further use (load), get artefact URI using `get_artifact_uri`

In [ ]:
import json

import mlflavors
import mlflow
from sktime.datasets import load_longley
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import (
    mean_absolute_error,
    mean_absolute_percentage_error,
)


ARTIFACT_PATH = "model"

with mlflow.start_run() as run:
    y, X = load_longley()
    y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

    forecaster = NaiveForecaster()
    forecaster.fit(
        y_train,
        X=X_train,
        fh=[1, 2, 3, 4],
    )

    # Extract parameters
    parameters = forecaster.get_params()

    # Evaluate model
    y_pred = forecaster.predict(X=X_test)
    metrics = {
        "mae": mean_absolute_error(y_test, y_pred),
        "mape": mean_absolute_percentage_error(y_test, y_pred),
    }

    print(f"Parameters: \n{json.dumps(parameters, indent=2)}")
    print(f"\nMetrics: \n{json.dumps(metrics, indent=2)}")

    # Log parameters and metrics
    mlflow.log_params(parameters)
    mlflow.log_metrics(metrics)

    # Log model to MLflow tracking server
    mlflavors.sktime.log_model(
        sktime_model=forecaster,
        artifact_path=ARTIFACT_PATH,
    )
    
    # Return model uri from the current run
    model_uri = mlflow.get_artifact_uri(ARTIFACT_PATH)
    
# Print the run id wich is used below for serving the model to a local REST API endpoint
print(f"\nMLflow run id:\n{run.info.run_id}")

## Viewing the model in the MLflow UI
To view the run output in the MLflow UI run the following command:

```bash
mlflow ui
```

When opening the MLflow runs detail page the serialized model artifact will show up, such as:

![title](../images/tracking_artifact_ui.png)

## Loading the model from MLflow

two options to load and predict:

* load in native format using `load_model`, then call method directly
* using `pyfunc.load_model` and `predict` with a `DataFrame` configuration to address method

option 1: `load_model`, native

In [ ]:
loaded_model = mlflavors.sktime.load_model(model_uri=model_uri)
print(loaded_model.predict_interval(fh=[1, 2, 3], X=X_test, coverage=[0.9, 0.95]))

option 2: `pyfunc` based

In [ ]:
import pandas as pd

# Convert test data to numpy array so it can be passed to pyfunc predict using
# a single-row Pandas DataFrame configuration argument
X_test_array = X_test.to_numpy()

# Create configuration DataFrame for interval forecast with nominal coverage
# value [0.9,0.95], future forecast horizon of 3 periods, and exogenous regressor.
predict_conf = pd.DataFrame(
    [
        {
            "fh": [1, 2, 3],
            "predict_method": "predict_interval",
            "coverage": [0.9, 0.95],
            "X": X_test_array,
        }
    ]
)

loaded_pyfunc = mlflavors.sktime.pyfunc.load_model(model_uri=model_uri)
print(loaded_pyfunc.predict(predict_conf))

# Serving the model to an endpoint

for serving at **local REST API endpoint**:

```bash
mlflow models serve -m runs:/<run_id>/model --env-manager local --host 127.0.0.1
```

with `run_id` as obtained in the "save" step.

Then, run the below model scoring script to request a prediction from the served model.

for serving the model to an **endpoint in the cloud** (e.g. Azure ML, AWS SageMaker, etc.):

use [MLflow deployment tools](https://mlflow.org/docs/latest/models.html#built-in-deployment-tools)):

In [ ]:
import pandas as pd
import requests
from sktime.datasets import load_longley
from sktime.forecasting.model_selection import temporal_train_test_split

y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

# Define local host and endpoint url
host = "127.0.0.1"
url = f"http://{host}:5000/invocations"

# Model scoring via REST API requires transforming the configuration DataFrame
# into JSON format. As numpy ndarray type is not JSON serializable we need to
# convert the exogenous regressor into a list. The wrapper instance will convert
# the list back to ndarray type as required by sktime predict methods. For more
# details read the MLflow deployment API reference.
# (https://mlflow.org/docs/latest/models.html#deploy-mlflow-models)
X_test_list = X_test.to_numpy().tolist()
predict_conf = pd.DataFrame(
    [
        {
            "fh": [1, 2, 3],
            "predict_method": "predict_interval",
            "coverage": [0.9, 0.95],
            "X": X_test_list,
        }
    ]
)

# Create dictionary with pandas DataFrame in the split orientation
json_data = {"dataframe_split": predict_conf.to_dict(orient="split")}

# Score model
response = requests.post(url, json=json_data)
print(response.json())

---
### Credits: notebook 6 - deploy to production with mlflow / mlflavors

notebook creation: benjaminbluhm

minor rearranging by fkiraly

mlflavors, `sktime` mlflow interface: benjaminbluhm